In [ ]:
#Importing libraries
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('../input/reducing-commercial-aviation-fatalities/train.csv')

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
# !pip install pycaret

In [ ]:
df['experiment'].value_counts()

In [ ]:
df['seat'].value_counts()

In [ ]:
df.describe(include="all")

In [ ]:
df.loc[df.experiment=='CA']['event'].value_counts()

In [ ]:
df.loc[df.experiment=='SS']['event'].value_counts()

In [ ]:
df.loc[df.experiment=='DA']['event'].value_counts()

In [ ]:
df.dtypes

In [ ]:
# select numerical column from data set
num_col=df.select_dtypes("number")

In [ ]:
num_col.head()

In [ ]:
cat_col=df.select_dtypes(exclude="number")

In [ ]:
cat_col

In [ ]:
cat_col.head()

In [ ]:
# plot categorical varible 
cat_col["experiment"].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.countplot(df["experiment"])
plt.show()

In [ ]:
pd.DataFrame(cat_col["event"].value_counts())

In [ ]:
pd.DataFrame((cat_col["event"].value_counts()/df.shape[0])*100)

In [ ]:
sns.countplot(df["event"])
plt.show()

In [ ]:
# Check the outliers in dataframe

In [ ]:
num_col.columns

In [ ]:
num_col.drop(["crew","seat"],axis=1,inplace=True)

In [ ]:
# Chcek the outliers in data 

In [ ]:
for i in num_col.columns:
#     print(df[i])
    sns.boxplot(x=num_col[i])      
    plt.show()

In [ ]:
def remove_outlier(df,col):
    q1=np.percentile(df[col],25)

    q3=np.percentile(df[col],75)

    iqr = q3 - q1
    upper_bound = q3 + (1.5 * iqr) 
    lower_bound = q1 - (1.5 * iqr)

    upper_value = np.percentile(df[col],90)
    lower_value = np.percentile(df[col],10)

    df.loc[df[col] >= upper_bound, col] = upper_value 
    df.loc[df[col] <= lower_bound, col] = lower_value
    return df

In [ ]:
for i in num_col:
    df1=remove_outlier(df,i)

In [ ]:
# After removing the outliers data
df1.head()

In [ ]:
df2=df1.drop(["crew","experiment","seat","event"],axis=1)
df2.head()

In [ ]:
for i in df2.columns:
    sns.boxplot(x=df[i])      
    plt.show()

In [ ]:
df2.head()

In [ ]:
# Scale the data 

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# create a scaler object
scaler = MinMaxScaler()
# fit and transform the data
df_norm = pd.DataFrame(scaler.fit_transform(df2), columns=df2.columns)

df_norm

In [ ]:
df5=pd.concat([df_norm,df[["crew","experiment","seat","event"]]],axis=1)

In [ ]:
df5.shape

In [ ]:
df5.head()

In [ ]:
df5.info()

In [ ]:
# label encoding for the categorical varible experiment.
from sklearn.preprocessing import LabelEncoder

In [ ]:
lab_enc = LabelEncoder()

In [ ]:
# Encode labels in column 'Country'. 
df5['experiment']= lab_enc.fit_transform(df5['experiment']) 

In [ ]:
df5.head()

In [ ]:
# suffle data 
from sklearn.utils import shuffle
df5 = shuffle(df5)

In [ ]:
df5.head()

In [ ]:
cat_col1=df5[["crew","experiment","seat","event"]]

In [ ]:
from scipy.stats import chi2_contingency

In [ ]:
# To check the relation between the target varible and other categorical varible
for i in cat_col1:
    print(pd.crosstab(cat_col1[i], cat_col1["event"]))
    result=chi2_contingency(pd.crosstab(cat_col1[i], cat_col1["event"]))
    print(f'P-value {result[1]}')
    print("\n")
    if result[1]< 0.05:
        print(f"Variable {i} and event are dependent on each other.")
        print("\n")
    else:
        print(f"Variable {i} and event are independent on each other.")
        print("\n")
    print("-------------------------------------------------------------")
    print("-------------------------------------------------------------")

In [ ]:
# from above result the the seat variable is not important so we can delete from dataset.
df5.drop(["seat"],axis=1,inplace=True)

In [ ]:
df5.head()

In [ ]:
# Bivariate analysis
# Here plot boxplot for categorical target variable continous target variable.

In [ ]:
for i in df5.columns[:-3]:
    print(i)

In [ ]:
for i in df5.columns[:-3]:
    sns.boxplot( y=df5[i], x=df["event"] );
    plt.show()

In [ ]:
# From above graph say that the all data are equally distibuted for all target category.

In [ ]:
x=df5.drop(["event"],axis=1)
y=df["event"]

In [ ]:
Counter(y)

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter

In [ ]:
# transform the dataset
oversample = SMOTE()
x, y = oversample.fit_resample(x, y)
# summarize distribution
counter = Counter(y)